# Action1	

"使用任何神经网络框架，对CIFAR-10进行分类

http://www.cs.toronto.edu/~kriz/cifar.html

训练集 50000，测试集 10000
图像大小 32*32 彩色
10个分类：ariplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck"			

"1、完成代码（30points）

2、使用ResNet, DenseNet或其他网络（20points）

3、Accuracy >90% （20points）"!

In [15]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Device 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
num_epochs = 80
learning_rate = 0.005

# Image preprocessing modules
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

# loading data
path = '/content/drive/My Drive/Kaikeba/名企/week7作业/data'
# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root=path,
                                             train=True, 
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root=path,
                                            train=False, 
                                            transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100, 
                                          shuffle=False)

# 3x3 convolution
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, bias=False)

# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

# ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)

    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

model = ResNet(ResidualBlock, [2, 2, 2]).to(device)


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# updating learning rate
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

# Train 
total_step = len(train_loader)
curr_lr = learning_rate
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 500 == 0:
            print ("Epoch [{}/{}], Loss: {:.4f}"
                   .format(epoch+1, num_epochs, loss.item()))

    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

# Test
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('/n 测试集Accuracy: {} %'.format(100 * correct / total))

# Save the model 
torch.save(model.state_dict(), 'resnet.ckpt')

Files already downloaded and verified
Epoch [1/80], Loss: 1.5767
Epoch [2/80], Loss: 1.0715
Epoch [3/80], Loss: 0.8666
Epoch [4/80], Loss: 0.7299
Epoch [5/80], Loss: 0.6891
Epoch [6/80], Loss: 0.5035
Epoch [7/80], Loss: 0.6563
Epoch [8/80], Loss: 0.5668
Epoch [9/80], Loss: 0.4184
Epoch [10/80], Loss: 0.4707
Epoch [11/80], Loss: 0.3670
Epoch [12/80], Loss: 0.4612
Epoch [13/80], Loss: 0.3226
Epoch [14/80], Loss: 0.2995
Epoch [15/80], Loss: 0.3233
Epoch [16/80], Loss: 0.2661
Epoch [17/80], Loss: 0.2828
Epoch [18/80], Loss: 0.3548
Epoch [19/80], Loss: 0.3978
Epoch [20/80], Loss: 0.2757
Epoch [21/80], Loss: 0.1242
Epoch [22/80], Loss: 0.1912
Epoch [23/80], Loss: 0.2567
Epoch [24/80], Loss: 0.2876
Epoch [25/80], Loss: 0.2786
Epoch [26/80], Loss: 0.2184
Epoch [27/80], Loss: 0.3037
Epoch [28/80], Loss: 0.2637
Epoch [29/80], Loss: 0.1395
Epoch [30/80], Loss: 0.1188
Epoch [31/80], Loss: 0.1977
Epoch [32/80], Loss: 0.2768
Epoch [33/80], Loss: 0.2956
Epoch [34/80], Loss: 0.1546
Epoch [35/80], Loss